In [14]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import keras
from numpy import array

import matplotlib.pyplot as plt
import math as mt

from keras.models import Sequential
from keras.layers import Dense , Dropout , Flatten
from keras.layers import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras import backend as K
K.set_image_dim_ordering('th')

# Root directory of the project
ROOT_DIR = os.getcwd()
DEEPFASHION_DATA_DIR = os.path.join(ROOT_DIR, "fashion_data")


# Deepfashion dataset을 load하고,  필요한 데이타를 메모리에 올립니다.
Deepfashion dataset에서 category, imagepath, bbox 데이타를 load한다.
txt파일들은 모두 첫번째 라인은 list 개수가 적혀 있고, 두번째 라인은 데이타 항목이 적혀 있다.
numpy에서 matrix로 load하기 위해 첫번째 라인과 두번째라인 삭제한다. (수동으로)

In [76]:
# category
val_category = np.genfromtxt(os.path.join(DEEPFASHION_DATA_DIR, "Anno", "list_category_img.txt"))
# string과 number가 섞여 있기 때문에 첫번째 인자가 non으로 오므로 삭제한다.
val_category = np.delete(val_category, 0, 1)
print(val_category)

# bbox
val_bbox = np.genfromtxt(os.path.join(DEEPFASHION_DATA_DIR, "Anno", "list_bbox.txt"))
# string과 number가 섞여 있기 때문에 첫번째 인자가 non으로 오므로 삭제한다.
val_bbox = np.delete(val_bbox, 0, 1)
print(val_bbox)

# image path
tmp_imagepath = []
with open(os.path.join(DEEPFASHION_DATA_DIR, "Anno", "list_category_img.txt")) as list_category_img:
    for row in list_category_img:
        imagepath = row.strip()[:-1].strip().replace(' ', '_')
        tmp_imagepath.append(imagepath)
        
val_imagepath = array(tmp_imagepath)
val_imagepath = val_imagepath.reshape((-1, 1))
print(val_imagepath)


# image path index
val_imagepath_idx = np.arange(val_imagepath.size)
val_imagepath_idx = val_imagepath_idx.reshape((-1, 1))
print(val_imagepath_idx)


print(val_imagepath.shape, val_imagepath_idx.shape, val_category.shape, val_bbox.shape);

[[ 3.]
 [ 3.]
 [ 3.]
 ...
 [41.]
 [41.]
 [41.]]
[[ 72.  79. 232. 273.]
 [ 67.  59. 155. 161.]
 [ 65.  65. 156. 200.]
 ...
 [ 18.  41. 149. 230.]
 [ 75.  47. 220. 300.]
 [ 36.  63. 202. 296.]]
[['img/Sheer_Pleated-Front_Blouse/img_00000001.jpg']
 ['img/Sheer_Pleated-Front_Blouse/img_00000002.jpg']
 ['img/Sheer_Pleated-Front_Blouse/img_00000003.jpg']
 ...
 ['img/Paisley_Print_Babydoll_Dress/img_00000052.jpg______________________4']
 ['img/Paisley_Print_Babydoll_Dress/img_00000053.jpg______________________4']
 ['img/Paisley_Print_Babydoll_Dress/img_00000054.jpg______________________4']]
[[     0]
 [     1]
 [     2]
 ...
 [289219]
 [289220]
 [289221]]
(289222, 1) (289222, 1) (289222, 1) (289222, 4)


# Make Dataset for ML
1.Load한 data들의 Array를 하나의 Array로 합친다.
2.shuffle한다.
3.80%는 train set으로 20%는 test set으로 분류한다.
    x_train, y_train, x_test, y_test 으로 저장한다.

In [109]:
dataset = np.concatenate((val_imagepath_idx, val_bbox, val_category), axis = 1) 
np.random.shuffle(dataset)

# 80%는 train set으로 20%는 test set으로 분류
dataset_threshold = 231377;
train_dataset = dataset[:dataset_threshold,]
test_dataset = dataset[dataset_threshold:,]

train_x = train_dataset[:,:5]
train_y = train_dataset[:,5:]

test_x = test_dataset[:,:5]
test_y = test_dataset[:,5:]